논문번호 | 대학 | 학과 | 전공 | 재직여부 | 성명 | 교번 | 풀네임 | Year | Title |Source | Status | DB

In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame as df
from openpyxl import load_workbook
import math

pd.options.display.float_format = '{:.0f}'.format
pd.options.display.max_columns = None

pip_list = pd.read_excel("DB/PIP/PIP_2014_2020.xlsx")
scopus_list = pd.read_excel("DB/SCOPUS/PI")
authors = pd.read_excel()